In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_csv('Nba_stats.csv' ,encoding= "utf-8", index_col=0 )
df['Player'] = df['Player'].str.encode('latin1').str.decode('utf-8')
df['FG%'] = df['FG%'].fillna(0)
df['FT%'] = df['FT%'].fillna(0)
df['3P%'] = df['3P%'].fillna(0)
df['2P%'] = df['2P%'].fillna(0)
df['eFG%'] = df['eFG%'].fillna(0)
df['Age'] = df['Age'].fillna(0)
df['Age'] = df['Age'].astype(int)
df['G'] = df['G'].fillna(0)
df['GS'] = df['GS'].fillna(0)
df['G'] = df['G'].astype(int)
df['GS'] = df['GS'].astype(int)

df['team_code'] = df['Team'].astype("category").cat.codes
df['pos_code'] = df['Pos'].astype('category').cat.codes
df['player_id'] = df['Player'].astype('category').cat.codes
df = df.sort_values(['Player', 'Season']).reset_index(drop=True)
df = df.drop_duplicates(subset=['Season', 'Player'], keep='first')




In [3]:
def next_season_stats(player):
   
    player[['Next-PPG', 'Next-APG', 'Next-RPG', 'Next-STL','Next-BLK' ]] = player[['PTS', 'AST', 'TRB', 'STL' , 'BLK']].shift(-1)
    return player

df = df.groupby('Player', group_keys=False).apply(next_season_stats)
#used to filter out inactive players by finding their last season played
df['last_season'] = df.groupby('Player')['Season'].transform('max')
#dropping any player that did not play last season
df = df.drop(df[df['last_season'] != 2025].index)

df

C:\Users\LL205\AppData\Local\Temp\ipykernel_7380\4022992992.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Player', group_keys=False).apply(next_season_stats)


,Player,Season,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,key,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,team_code,pos_code,player_id,Next-PPG,Next-APG,Next-RPG,Next-STL,Next-BLK,last_season
0,A.J. Green,2023,23,MIL,SG,35,1,9.9,1.5,3.6,0.424,1.3,3.0,0.419,0.3,0.6,0.450,0.600,0.1,0.1,1.000,0.2,1.1,1.3,0.6,0.2,0.0,0.3,0.9,4.4,A.J. Green_2023,11.5,0.607,0.840,0.032,1.9,11.3,6.8,8.7,0.8,0.0,6.6,16.6,0.5,0.3,0.8,0.111,-0.3,-0.6,-0.9,0.1,19,4,0,4.5,0.5,1.1,0.2,0.1,2025
1,A.J. Green,2024,24,MIL,SG,56,0,11.0,1.5,3.5,0.423,1.2,3.0,0.408,0.3,0.5,0.519,0.599,0.3,0.3,0.895,0.2,1.0,1.1,0.5,0.2,0.1,0.2,0.9,4.5,A.J. Green_2024,10.5,0.617,0.862,0.097,1.6,9.6,5.7,6.4,0.7,0.6,5.5,15.2,0.7,0.3,1.1,0.084,-0.4,-1.7,-2.1,0.0,19,4,0,7.4,1.5,2.4,0.5,0.1,2025
2,A.J. Green,2025,25,MIL,SG,73,7,22.7,2.5,5.8,0.429,2.1,5.0,0.427,0.4,0.8,0.443,0.612,0.3,0.4,0.815,0.2,2.1,2.4,1.5,0.5,0.1,0.5,2.2,7.4,A.J. Green_2025,9.2,0.621,0.856,0.064,1.2,9.8,5.7,8.5,1.1,0.4,8.4,12.6,1.6,1.3,2.8,0.082,-1.7,-0.4,-2.2,-0.1,19,4,0,NaN,NaN,NaN,NaN,NaN,2025
4,A.J. Lawson,2023,22,2TM,SG,15,0,7.2,1.5,2.9,0.500,0.7,1.7,0.400,0.8,1.3,0.632,0.614,0.1,0.5,0.250,0.4,1.0,1.4,0.1,0.1,0.0,0.2,0.7,3.7,A.J. Lawson_2023,13.9,0.589,0.568,0.182,6.4,16.3,11.3,3.0,0.9,0.0,5.9,21.1,0.1,0.1,0.1,0.063,-1.5,-2.9,-4.4,-0.1,0,4,2,3.2,0.5,1.2,0.2,0.1,2025
13,A.J. Lawson,2024,23,DAL,SG,42,0,7.4,1.3,2.9,0.446,0.3,1.2,0.260,1.0,1.7,0.577,0.500,0.4,0.5,0.652,0.3,0.9,1.2,0.5,0.2,0.1,0.3,0.5,3.2,A.J. Lawson_2024,11.2,0.519,0.413,0.190,4.9,12.6,8.8,8.9,1.5,0.8,9.6,20.0,0.0,0.3,0.2,0.036,-3.1,-1.5,-4.6,-0.2,9,4,2,9.1,1.2,3.3,0.5,0.2,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12413,Zion Williamson,2021,20,NOP,PF,61,61,33.2,10.4,17.0,0.611,0.2,0.6,0.294,10.2,16.4,0.622,0.616,6.0,8.7,0.698,2.7,4.5,7.2,3.7,0.9,0.6,2.7,2.2,27.0,Zion Williamson_2021,27.1,0.649,0.033,0.510,8.9,14.7,11.8,19.7,1.3,1.8,11.6,29.8,7.1,1.6,8.7,0.205,6.1,-0.3,5.8,4.0,21,1,1493,26.0,4.6,7.0,1.1,0.6,2025
12414,Zion Williamson,2023,22,NOP,PF,29,29,33.0,9.8,16.2,0.608,0.2,0.7,0.368,9.6,15.5,0.618,0.615,6.1,8.6,0.714,2.0,5.0,7.0,4.6,1.1,0.6,3.4,2.2,26.0,Zion Williamson_2023,25.2,0.652,0.041,0.529,6.9,17.0,12.0,24.4,1.6,1.6,14.6,30.4,2.3,1.2,3.6,0.180,4.8,1.0,5.8,1.9,21,1,1493,22.9,5.0,5.8,1.1,0.7,2025
12415,Zion Williamson,2024,23,NOP,PF,70,70,31.5,8.9,15.6,0.570,0.1,0.3,0.333,8.8,15.4,0.574,0.573,5.0,7.1,0.702,1.7,4.1,5.8,5.0,1.1,0.7,2.8,2.3,22.9,Zion Williamson_2024,22.8,0.610,0.016,0.452,6.2,14.2,10.2,26.5,1.7,2.1,12.8,29.7,4.7,2.8,7.5,0.164,3.3,0.6,3.8,3.3,21,1,1493,24.6,5.3,7.2,1.2,0.9,2025
12416,Zion Williamson,2025,24,NOP,PF,30,30,28.6,9.6,16.9,0.567,0.1,0.4,0.231,9.5,16.5,0.576,0.570,5.3,8.0,0.656,2.5,4.7,7.2,5.3,1.2,0.9,3.0,2.7,24.6,Zion Williamson_2025,27.3,0.600,0.026,0.474,9.3,18.2,13.6,36.6,2.1,3.0,12.8,34.7,1.9,0.6,2.5,0.141,5.9,1.0,7.0,1.9,21,1,1493,NaN,NaN,NaN,NaN,NaN,2025


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split


In [5]:
base_model = RandomForestRegressor()
model = MultiOutputRegressor(base_model)


In [6]:
features = ['G', 'GS','PTS' ,'AST', 'TRB', 'Age', 'STL', 'BLK', 'MP', '3PA', 'team_code', 'FGA', 'pos_code','player_id']



In [7]:
years = sorted(df["Season"].unique())
years

[np.int64(2015),
 np.int64(2016),
 np.int64(2017),
 np.int64(2018),
 np.int64(2019),
 np.int64(2020),
 np.int64(2021),
 np.int64(2022),
 np.int64(2023),
 np.int64(2024),
 np.int64(2025)]

In [8]:
def backtest(data, model, features, start=3, step=1):
    all_predictions = []
    years = sorted(data["Season"].unique())


    prediction = ['P-PPG', 'P-APG', 'P-RBG', 'P-STL', 'P-BLK']
    
    actual_targets = ['Next-PPG','Next-APG','Next-RPG','Next-STL','Next-BLK']

    for i in range(start, len(years), step):
        current_year = years[i]
        
        train = data[data["Season"] < current_year]
        test = data[data["Season"] == current_year]
        
        model.fit(train[features], train[actual_targets])
        preds = model.predict(test[features]) 
        
        preds_df = pd.DataFrame(preds, columns=prediction ,index=test.index)
        preds_df = preds_df[prediction].round(1)
        combined = pd.concat([test[['Player', 'Season']], test[actual_targets], preds_df], axis= 1)
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)
   


In [9]:
predictions = backtest(df, model, features)



In [10]:
predictions


,Player,Season,Next-PPG,Next-APG,Next-RPG,Next-STL,Next-BLK,P-PPG,P-APG,P-RBG,P-STL,P-BLK
49,Aaron Gordon,2018,16.0,3.7,7.4,0.7,0.7,19.0,2.3,8.7,0.9,1.1
125,Al Horford,2018,13.6,4.2,6.7,0.9,1.3,12.1,5.7,7.2,0.7,1.3
162,Alec Burks,2018,8.8,2.0,3.7,0.6,0.3,10.5,2.1,3.8,0.7,0.2
217,Alex Caruso,2018,9.2,3.1,2.7,1.0,0.4,6.4,3.0,3.0,0.8,0.5
239,Alex Len,2018,11.1,1.1,5.5,0.4,0.9,8.4,1.1,7.5,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12369,Zach LaVine,2025,NaN,NaN,NaN,NaN,NaN,21.4,3.9,4.3,0.8,0.2
12403,Zeke Nnaji,2025,NaN,NaN,NaN,NaN,NaN,4.9,0.7,3.0,0.4,0.7
12411,Ziaire Williams,2025,NaN,NaN,NaN,NaN,NaN,10.0,1.5,4.5,1.0,0.5
12416,Zion Williamson,2025,NaN,NaN,NaN,NaN,NaN,25.8,5.4,7.3,1.1,0.8


In [11]:
from sklearn.metrics import mean_squared_error
mask = predictions['Season'] < 2025
mse = mean_squared_error(predictions.loc[mask , ['Next-PPG','Next-APG','Next-RPG','Next-STL','Next-BLK']], predictions.loc[mask, ['P-PPG', 'P-APG', 'P-RBG', 'P-STL', 'P-BLK']])
print(mse) 

predictions[['P-PPG', 'P-APG', 'P-RBG', 'P-STL', 'P-BLK']].describe()

2.5414834099030115


,P-PPG,P-APG,P-RBG,P-STL,P-BLK
count,2528.000000,2528.000000,2528.000000,2528.000000,2528.000000
mean,11.678283,2.691970,4.514557,0.785799,0.510918
std,6.188820,1.837774,2.301487,0.316691,0.386167
min,2.300000,0.400000,1.200000,0.200000,0.100000
25%,6.900000,1.400000,2.900000,0.600000,0.300000
50%,9.800000,2.000000,4.000000,0.700000,0.400000
75%,15.400000,3.525000,5.300000,1.000000,0.700000
max,32.600000,10.400000,14.500000,1.900000,3.100000
